In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import metrics

# Step 1: Data Preprocessing
df = pd.read_csv('data/genre.csv')  # Read the dataset

# limit the dataset to 2000 images
df = df[:2000]

image_paths = df['imdbID']  # Extract image paths
labels = df.iloc[:, 3:].values  # Extract label values

image_size = (450, 300)  # Define the desired image size

def preprocess_image(image_path):
    path = "data/clean_images/" + image_path + ".jpg"
    image = load_img(path)
    image = img_to_array(image)
    image = image / 255.0  # Normalize pixel values
    return image

images = np.array([preprocess_image(path) for path in image_paths])

# Step 2: Split the Dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Step 3: Build the Model
num_classes = labels.shape[1]  # Number of classes

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Step 4: Compile the Model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       metrics.Precision(name='precision'),
                       metrics.Recall(name='recall')])

# Step 5: Train the Model
batch_size = 16
epochs = 50

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Step 6: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Step 7: Predictions
predictions = model.predict(X_test)

# Set a threshold to determine the labels based on the output probabilities
threshold = 0.5
predicted_labels = np.where(predictions > threshold, 1, 0)

# Print the first 10 images and their predicted labels
for i in range(10):
    print(f'Image {i+1}:')
    print(f'Predicted Labels: {predicted_labels[i]}')
    print(f'Actual Labels: {y_test[i]}')
    print()

# Step 8: Save the Model
model.save('models/genre_model.h5')


Epoch 1/50
 19/100 [====>.........................] - ETA: 1:18 - loss: 0.3762 - accuracy: 0.2171 - precision: 0.2902 - recall: 0.2417

KeyboardInterrupt: 